### Import statements

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
from sklearn.metrics import classification_report, confusion_matrix
from src.utils import DatasetUtils

c:\users\aakaas~1\docume~1\nyu\projects\speech~1\env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Defining system constants

In [2]:
keras = tf.keras
py.init_notebook_mode(connected=True)
classes = DatasetUtils().get_label_encoder().classes_

### Loading data

#### Loading test data

In [3]:
test_data, test_labels = DatasetUtils().get_dataset_and_encoded_labels('test_data.npy', 'test_labels.npy')
test_data_unrolled = np.load('./data/test_labels_unrolled.npy')

## Models trained on clean data

#### Loading models and the training history

In [4]:
c_cnn = keras.models.load_model('./models/clean/models/convnet.h5')
c_resnet = keras.models.load_model('./models/clean/models/resnet.h5')
c_rrnn = keras.models.load_model('./models/clean/models/residualresnet.h5')

In [5]:
c_cnn_history = pd.read_csv('./models/clean/models/convnet.csv').set_index('epoch')
c_resnet_history = pd.read_csv('./models/clean/models/resnet.csv').set_index('epoch')
c_rrnn_history = pd.read_csv('./models/clean/models/residualresnet.csv').set_index('epoch')

### Architectures

In [6]:
print('CNN Architecture:')
c_cnn.summary()
print('ResNet Architecture:')
c_resnet.summary()
print('RRNN Architecture:')
c_rrnn.summary()

CNN Architecture:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 63, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 63, 48)       1200      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 63, 48)       192       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 63, 48)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 63, 48)       55344     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 63, 48)       192       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 63, 48)       0  

### Evaluating the models on test data

In [7]:
print('CNN: test loss: {} - test accuracy: {}'.format(*c_cnn.evaluate(test_data, test_labels, verbose=0)))
print('Resnet: test loss: {} - test accuracy: {}'.format(*c_resnet.evaluate(test_data, test_labels, verbose=0)))
print('RRNN: test loss: {} - test accuracy: {}'.format(*c_rrnn.evaluate(test_data, test_labels, verbose=0)))

CNN: test loss: 0.674249774605668 - test accuracy: 0.8149231894834249
Resnet: test loss: 0.5840792138273906 - test accuracy: 0.8504754937384019
RRNN: test loss: 0.6701930581346078 - test accuracy: 0.8339429407025569


### Predictions

#### Getting labels for the predicted data and test data

In [8]:
c_cnn_labels = classes[np.argmax(c_cnn.predict(test_data), axis=-1)]
c_resnet_labels = classes[np.argmax(c_resnet.predict(test_data), axis=-1)]
c_rrnn_labels = classes[np.argmax(c_rrnn.predict(test_data), axis=-1)]
truth_labels = classes[np.argmax(test_labels, axis=-1)]

### Classification Reports

In [9]:
print('CNN Classification Report:')
print(classification_report(truth_labels, c_cnn_labels))
print('ResNet Classification Report:')
print(classification_report(truth_labels, c_resnet_labels))
print('RRNN Classification Report:')
print(classification_report(truth_labels, c_rrnn_labels))

CNN Classification Report:
             precision    recall  f1-score   support

       down       0.87      0.74      0.80       253
      eight       0.91      0.80      0.85       257
       five       0.84      0.79      0.81       271
       four       0.96      0.74      0.84       253
         go       0.92      0.57      0.71       251
       left       0.87      0.78      0.82       267
       nine       0.90      0.86      0.88       259
         no       0.91      0.68      0.78       252
        off       0.80      0.81      0.81       262
         on       0.90      0.91      0.90       246
        one       0.99      0.81      0.89       248
      right       0.97      0.76      0.85       259
      seven       0.97      0.82      0.89       239
        six       0.88      0.81      0.84       244
       stop       0.90      0.84      0.87       249
      three       0.96      0.49      0.65       267
        two       0.93      0.73      0.82       264
    unknown       

### Confusion matrix

In [10]:
cm = confusion_matrix(truth_labels, c_cnn_labels)
cm = np.divide(cm, cm.sum(axis=1)) * 100
py.iplot(dict(
    data=[go.Heatmap(z=cm, x=classes, y=classes)],
    layout=go.Layout(title='Confusion Matrix for CNN')))

In [11]:
cm = confusion_matrix(truth_labels, c_resnet_labels)
cm = np.divide(cm, cm.sum(axis=1)) * 100
py.iplot(dict(
    data=[go.Heatmap(z=cm, x=classes, y=classes)],
    layout=go.Layout(title='Confusion Matrix for ResNet')))

In [12]:
cm = confusion_matrix(truth_labels, c_rrnn_labels)
cm = np.divide(cm, cm.sum(axis=1)) * 100
py.iplot(dict(
    data=[go.Heatmap(z=cm, x=classes, y=classes)],
    layout=go.Layout(title='Confusion Matrix for RRNN')))

### Error matrix

In [13]:
cm = confusion_matrix(truth_labels, c_cnn_labels)
cm = np.divide(cm, cm.sum(axis=1)) * 100
np.fill_diagonal(cm, 0)
py.iplot(dict(
    data=[go.Heatmap(z=cm, x=classes, y=classes)],
    layout=go.Layout(title='Error Matrix for CNN')))

In [14]:
cm = confusion_matrix(truth_labels, c_resnet_labels)
cm = np.divide(cm, cm.sum(axis=1)) * 100
np.fill_diagonal(cm, 0)
py.iplot(dict(
    data=[go.Heatmap(z=cm, x=classes, y=classes)],
    layout=go.Layout(title='Error Matrix for ResNet')))

In [15]:
cm = confusion_matrix(truth_labels, c_rrnn_labels)
cm = np.divide(cm, cm.sum(axis=1)) * 100
np.fill_diagonal(cm, 0)
py.iplot(dict(
    data=[go.Heatmap(z=cm, x=classes, y=classes)],
    layout=go.Layout(title='Error Matrix for RRNN')))

### Intra-model Training History

In [16]:
py.iplot(dict(data=[
        go.Scatter(x=c_cnn_history.index, y=c_cnn_history.acc, mode='lines', name='Training accuracy'),
        go.Scatter(x=c_cnn_history.index, y=c_cnn_history.loss, mode='lines', name='Training loss'),
        go.Scatter(x=c_cnn_history.index, y=c_cnn_history.val_acc, mode='lines', name='Validation accuracy'),
        go.Scatter(x=c_cnn_history.index, y=c_cnn_history.val_loss, mode='lines', name='Validation loss')],
              layout=go.Layout(title='Training History for CNN',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

In [17]:
py.iplot(dict(data=[
        go.Scatter(x=c_resnet_history.index, y=c_resnet_history.acc, mode='lines', name='Training accuracy'),
        go.Scatter(x=c_resnet_history.index, y=c_resnet_history.loss, mode='lines', name='Training loss'),
        go.Scatter(x=c_resnet_history.index, y=c_resnet_history.val_acc, mode='lines', name='Validation accuracy'),
        go.Scatter(x=c_resnet_history.index, y=c_resnet_history.val_loss, mode='lines', name='Validation loss')],
              layout=go.Layout(title='Training History for ResNet',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

In [18]:
py.iplot(dict(data=[
        go.Scatter(x=c_rrnn_history.index, y=c_rrnn_history.acc, mode='lines', name='Training accuracy'),
        go.Scatter(x=c_rrnn_history.index, y=c_rrnn_history.loss, mode='lines', name='Training loss'),
        go.Scatter(x=c_rrnn_history.index, y=c_rrnn_history.val_acc, mode='lines', name='Validation accuracy'),
        go.Scatter(x=c_rrnn_history.index, y=c_rrnn_history.val_loss, mode='lines', name='Validation loss')],
              layout=go.Layout(title='Training History for RRNN',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

### Inter-model training history

In [19]:
py.iplot(dict(data=[
        go.Scatter(x=c_cnn_history.index, y=c_cnn_history.acc, mode='lines', name='CNN'),
        go.Scatter(x=c_resnet_history.index, y=c_resnet_history.loss, mode='lines', name='ResNet'),
        go.Scatter(x=c_rrnn_history.index, y=c_rrnn_history.val_acc, mode='lines', name='RRNN')],
              layout=go.Layout(title='Training Accuracy',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

In [20]:
py.iplot(dict(data=[
        go.Scatter(x=c_cnn_history.index, y=c_cnn_history.loss, mode='lines', name='CNN'),
        go.Scatter(x=c_resnet_history.index, y=c_resnet_history.loss, mode='lines', name='ResNet'),
        go.Scatter(x=c_rrnn_history.index, y=c_rrnn_history.loss, mode='lines', name='RRNN')],
              layout=go.Layout(title='Training Loss',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

In [21]:
py.iplot(dict(data=[
        go.Scatter(x=c_cnn_history.index, y=c_cnn_history.val_acc, mode='lines', name='CNN'),
        go.Scatter(x=c_resnet_history.index, y=c_resnet_history.val_acc, mode='lines', name='ResNet'),
        go.Scatter(x=c_rrnn_history.index, y=c_rrnn_history.val_acc, mode='lines', name='RRNN')],
              layout=go.Layout(title='Validation Accuracy',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

In [22]:
py.iplot(dict(data=[
        go.Scatter(x=c_cnn_history.index, y=c_cnn_history.val_loss, mode='lines', name='CNN'),
        go.Scatter(x=c_resnet_history.index, y=c_resnet_history.val_loss, mode='lines', name='ResNet'),
        go.Scatter(x=c_rrnn_history.index, y=c_rrnn_history.val_loss, mode='lines', name='RRNN')],
              layout=go.Layout(title='Validation Loss',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

## Models trained on noisy data

#### Loading models and the training history

In [24]:
n_cnn = keras.models.load_model('./models/noisy/models/convnet.h5')
n_resnet = keras.models.load_model('./models/noisy/models/resnet.h5')
n_rrnn = keras.models.load_model('./models/noisy/models/residualresnet.h5')

In [25]:
n_cnn_history = pd.read_csv('./models/noisy/models/convnet.csv').set_index('epoch')
n_resnet_history = pd.read_csv('./models/noisy/models/resnet.csv').set_index('epoch')
n_rrnn_history = pd.read_csv('./models/noisy/models/residualresnet.csv').set_index('epoch')

### Architectures

In [26]:
print('CNN Architecture:')
n_cnn.summary()
print('ResNet Architecture:')
n_resnet.summary()
print('RRNN Architecture:')
n_rrnn.summary()

CNN Architecture:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 63, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 63, 48)       1200      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 63, 48)       192       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 63, 48)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 63, 48)       55344     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 63, 48)       192       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 63, 48)       0  

### Evaluating the models on test data

In [27]:
print('CNN: test loss: {} - test accuracy: {}'.format(*n_cnn.evaluate(test_data, test_labels, verbose=0)))
print('Resnet: test loss: {} - test accuracy: {}'.format(*n_resnet.evaluate(test_data, test_labels, verbose=0)))
print('RRNN: test loss: {} - test accuracy: {}'.format(*n_rrnn.evaluate(test_data, test_labels, verbose=0)))

CNN: test loss: 0.5874986929403402 - test accuracy: 0.8414045354355489
Resnet: test loss: 0.6533726495552342 - test accuracy: 0.8320409655745394
RRNN: test loss: 0.7017751318002672 - test accuracy: 0.8260424287369762


### Predictions

#### Getting labels for the predicted data and test data

In [28]:
n_cnn_labels = classes[np.argmax(n_cnn.predict(test_data), axis=-1)]
n_resnet_labels = classes[np.argmax(n_resnet.predict(test_data), axis=-1)]
n_rrnn_labels = classes[np.argmax(n_rrnn.predict(test_data), axis=-1)]
truth_labels = classes[np.argmax(test_labels, axis=-1)]

### Classification Reports

In [29]:
print('CNN Classification Report:')
print(classification_report(truth_labels, n_cnn_labels))
print('ResNet Classification Report:')
print(classification_report(truth_labels, n_resnet_labels))
print('RRNN Classification Report:')
print(classification_report(truth_labels, n_rrnn_labels))

CNN Classification Report:
             precision    recall  f1-score   support

       down       0.89      0.78      0.83       253
      eight       0.96      0.79      0.87       257
       five       0.86      0.85      0.85       271
       four       0.94      0.81      0.87       253
         go       0.89      0.68      0.77       251
       left       0.92      0.84      0.87       267
       nine       0.92      0.90      0.91       259
         no       0.90      0.77      0.83       252
        off       0.85      0.81      0.83       262
         on       0.91      0.91      0.91       246
        one       0.99      0.80      0.89       248
      right       0.95      0.81      0.88       259
      seven       0.97      0.82      0.89       239
        six       0.92      0.78      0.85       244
       stop       0.90      0.87      0.88       249
      three       0.88      0.61      0.72       267
        two       0.95      0.74      0.83       264
    unknown       

### Confusion matrix

In [30]:
cm = confusion_matrix(truth_labels, n_cnn_labels)
cm = np.divide(cm, cm.sum(axis=1)) * 100
py.iplot(dict(
    data=[go.Heatmap(z=cm, x=classes, y=classes)],
    layout=go.Layout(title='Confusion Matrix for CNN')))

In [31]:
cm = confusion_matrix(truth_labels, n_resnet_labels)
cm = np.divide(cm, cm.sum(axis=1)) * 100
py.iplot(dict(
    data=[go.Heatmap(z=cm, x=classes, y=classes)],
    layout=go.Layout(title='Confusion Matrix for ResNet')))

In [32]:
cm = confusion_matrix(truth_labels, n_rrnn_labels)
cm = np.divide(cm, cm.sum(axis=1)) * 100
py.iplot(dict(
    data=[go.Heatmap(z=cm, x=classes, y=classes)],
    layout=go.Layout(title='Confusion Matrix for RRNN')))

### Error matrix

In [33]:
cm = confusion_matrix(truth_labels, n_cnn_labels)
cm = np.divide(cm, cm.sum(axis=1)) * 100
np.fill_diagonal(cm, 0)
py.iplot(dict(
    data=[go.Heatmap(z=cm, x=classes, y=classes)],
    layout=go.Layout(title='Error Matrix for CNN')))

In [34]:
cm = confusion_matrix(truth_labels, n_resnet_labels)
cm = np.divide(cm, cm.sum(axis=1)) * 100
np.fill_diagonal(cm, 0)
py.iplot(dict(
    data=[go.Heatmap(z=cm, x=classes, y=classes)],
    layout=go.Layout(title='Error Matrix for ResNet')))

In [35]:
cm = confusion_matrix(truth_labels, n_rrnn_labels)
cm = np.divide(cm, cm.sum(axis=1)) * 100
np.fill_diagonal(cm, 0)
py.iplot(dict(
    data=[go.Heatmap(z=cm, x=classes, y=classes)],
    layout=go.Layout(title='Error Matrix for RRNN')))

### Intra-model Training History

In [36]:
py.iplot(dict(data=[
        go.Scatter(x=n_cnn_history.index, y=n_cnn_history.acc, mode='lines', name='Training accuracy'),
        go.Scatter(x=n_cnn_history.index, y=n_cnn_history.loss, mode='lines', name='Training loss'),
        go.Scatter(x=n_cnn_history.index, y=n_cnn_history.val_acc, mode='lines', name='Validation accuracy'),
        go.Scatter(x=n_cnn_history.index, y=n_cnn_history.val_loss, mode='lines', name='Validation loss')],
              layout=go.Layout(title='Training History for CNN',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

In [37]:
py.iplot(dict(data=[
        go.Scatter(x=n_resnet_history.index, y=n_resnet_history.acc, mode='lines', name='Training accuracy'),
        go.Scatter(x=n_resnet_history.index, y=n_resnet_history.loss, mode='lines', name='Training loss'),
        go.Scatter(x=n_resnet_history.index, y=n_resnet_history.val_acc, mode='lines', name='Validation accuracy'),
        go.Scatter(x=n_resnet_history.index, y=n_resnet_history.val_loss, mode='lines', name='Validation loss')],
              layout=go.Layout(title='Training History for ResNet',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

In [38]:
py.iplot(dict(data=[
        go.Scatter(x=n_rrnn_history.index, y=n_rrnn_history.acc, mode='lines', name='Training accuracy'),
        go.Scatter(x=n_rrnn_history.index, y=n_rrnn_history.loss, mode='lines', name='Training loss'),
        go.Scatter(x=n_rrnn_history.index, y=n_rrnn_history.val_acc, mode='lines', name='Validation accuracy'),
        go.Scatter(x=n_rrnn_history.index, y=n_rrnn_history.val_loss, mode='lines', name='Validation loss')],
              layout=go.Layout(title='Training History for RRNN',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

### Inter-model training history

In [39]:
py.iplot(dict(data=[
        go.Scatter(x=n_cnn_history.index, y=n_cnn_history.acc, mode='lines', name='CNN'),
        go.Scatter(x=n_resnet_history.index, y=n_resnet_history.loss, mode='lines', name='ResNet'),
        go.Scatter(x=n_rrnn_history.index, y=n_rrnn_history.val_acc, mode='lines', name='RRNN')],
              layout=go.Layout(title='Training Accuracy',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

In [40]:
py.iplot(dict(data=[
        go.Scatter(x=n_cnn_history.index, y=n_cnn_history.loss, mode='lines', name='CNN'),
        go.Scatter(x=n_resnet_history.index, y=n_resnet_history.loss, mode='lines', name='ResNet'),
        go.Scatter(x=n_rrnn_history.index, y=n_rrnn_history.loss, mode='lines', name='RRNN')],
              layout=go.Layout(title='Training Loss',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

In [41]:
py.iplot(dict(data=[
        go.Scatter(x=n_cnn_history.index, y=n_cnn_history.val_acc, mode='lines', name='CNN'),
        go.Scatter(x=n_resnet_history.index, y=n_resnet_history.val_acc, mode='lines', name='ResNet'),
        go.Scatter(x=n_rrnn_history.index, y=n_rrnn_history.val_acc, mode='lines', name='RRNN')],
              layout=go.Layout(title='Validation Accuracy',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))

In [42]:
py.iplot(dict(data=[
        go.Scatter(x=n_cnn_history.index, y=n_cnn_history.val_loss, mode='lines', name='CNN'),
        go.Scatter(x=n_resnet_history.index, y=n_resnet_history.val_loss, mode='lines', name='ResNet'),
        go.Scatter(x=n_rrnn_history.index, y=n_rrnn_history.val_loss, mode='lines', name='RRNN')],
              layout=go.Layout(title='Validation Loss',
                               xaxis=dict(title='Epochs'),
                               yaxis=dict(title='Value'))))